In [ ]:
import odds_datasets
from iforest import BAD_IForest
import batch_bald
import numpy as np

for dataset in odds_datasets.datasets_names:
    X, y = odds_datasets.load(dataset)
    model = BAD_IForest(contamination=y.mean()).fit(X)
    print(f"\nDATASET:     {dataset[:10]}\tX: {X.shape}\tcontamination: {100*y.mean():.2f}%")
    for batch_size in [10]:
        print(f"\nBATCH SIZE: {batch_size}")

        # RANDOM
        print("random          \t", end="")
        %timeit -n 10 model.decision_function(X)

        # INDEPENDENT 
        for interest_method in ["bald", "margin", "anom"]:
            model.interest_method = interest_method # type: ignore
            print(f"independent {interest_method}\t", end="")
            %timeit -n 10 model.get_queries(X, batch_size, independent=True) 

        # WORSTCASE
        for interest_method in ["bald", "margin", "anom"]:
            model.interest_method = interest_method # type: ignore
            print(f"worstcase {interest_method}  \t", end="")
            %timeit -n 10 model.get_queries(X, batch_size, independent=False)

        # BATCHBALD
        print(f"batchbald       \t", end="")
        %timeit -n 10 batch_bald.get_queries(model=model, X=X, batch_size=batch_size)